In [1]:
import time
import torch
from torch import nn
from torch.optim import SGD
import copy
from torch.nn.utils.rnn import pack_padded_sequence

In [2]:
import csv

In [3]:
import pandas as pd

import numpy as np 

data_file = pd.read_csv('train', quoting=csv.QUOTE_NONE,sep=" ", names= ['index', 'word', 'tag'])

In [4]:
# convert 
data_array = data_file.to_numpy()

In [5]:
### store the lists of sentences into a big list 
 
def data_to_stream(data):
    data_stream = []
    i = 0 
    for index, word, tag in data:
        if index == 1:
            temp = []
            temp.append([index,word, tag])
            
        else:
            temp.append([index ,word,tag ])
            
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            data_stream.append(temp)
        
        i += 1
            
    return data_stream 

In [6]:
data_stream = data_to_stream(data_array)

In [7]:
def data_to_stream(data):
    data_stream = []
    i = 0 
    for index, word, tag in data:
        if index == 1:
            temp = []
            temp.append([index,word, tag])
            
        else:
            temp.append([index ,word,tag ])
            
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            data_stream.append(temp)
        
        i += 1
            
    return data_stream

In [8]:
def x_y(indexical_train):
    x = []
    y = []
    for sentence in indexical_train:
        temp_x = []
        temp_y = []
        for word in sentence:
            temp_x.append(word[1])
            temp_y.append(word[2])
        x.append(temp_x)
        y.append(temp_y)
    return x,y
        

In [9]:
train_word,train_tag = x_y(data_stream)

In [10]:
all_data_list = list(zip(train_word, train_tag ))

In [11]:
word_set = set([str(word) for seq in train_word for word in seq])

word_vocab = ['<pad>']+sorted(word_set)+['<unk>']

word_to_index = {word:idx for idx,word in enumerate(word_vocab)}

index_to_word = {idx:word for idx,word in enumerate(word_vocab)}

In [12]:
tag_set =set([str(tag)for sentence in train_tag for tag in sentence])
tag_vocab = ['<pad>'] + sorted(tag_set) 
tag_to_index = {tag:idx for idx,tag in enumerate(tag_vocab)}
index_to_tag = {idx:tag for idx,tag in enumerate(tag_vocab)}

In [13]:
def indexical_sentence_word(sentence,word_to_index):
    w=[]
    for word in sentence:
        if word in word_to_index:
            w.append(word_to_index[word])
        else:
            w.append(word_to_index['<unk>'])
    #w_tensor = torch.Tensor(w)
    return w
        
    

In [14]:
def indexical_sentence_tag(sentence,tag_to_index):
    t = []
    for tag in sentence:
        t.append(tag_to_index[tag])
    #t_tensor = torch.Tensor(t)
    return t

In [15]:
def indexical_x_y(all_data_list):
    x_train = []
    y_train = []
    for item in all_data_list:
        words,tags = item[0],item[1]
        seq_tensor = indexical_sentence_word(words,word_to_index)
        tags_tensor = indexical_sentence_tag(tags,tag_to_index)
        x_train.append(seq_tensor)
        y_train.append(tags_tensor)
    return x_train,y_train

In [16]:
tag_to_index

{'<pad>': 0,
 'B-LOC': 1,
 'B-MISC': 2,
 'B-ORG': 3,
 'B-PER': 4,
 'I-LOC': 5,
 'I-MISC': 6,
 'I-ORG': 7,
 'I-PER': 8,
 'O': 9}

In [17]:
x_train, y_train = indexical_x_y(all_data_list)

In [18]:
sentence_lenth = [len(sen) for sen in x_train]

longest = max(sentence_lenth)

In [19]:
word_pad_index = word_to_index['<pad>']
tag_pad_index = tag_to_index['<pad>']

In [20]:
dev_file = pd.read_csv('dev', quoting=csv.QUOTE_NONE, sep=" ", names= ['index', 'word', 'tag'])

In [21]:
#dev_file
# convert 
dev_array = dev_file.to_numpy()
dev_stream = data_to_stream(dev_array)


In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence


In [23]:
def list_to_array(list_data):
    data = copy.deepcopy(list_data)
    for i in range(len(data)):
        data[i] = torch.from_numpy(np.array(data[i]))
    return data

In [24]:
x_train = list_to_array(x_train)

In [25]:
y_train = list_to_array(y_train)

In [26]:
from torch.utils.data import Dataset, DataLoader
## refer to pytorch tutorials on how to inherit from Dataset class


def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=0)

    return xx_pad, yy_pad, x_lens, y_lens

dataset = list(zip(x_train, y_train))
data_loader = DataLoader(dataset=dataset, batch_size=16, shuffle=False, collate_fn=pad_collate)


In [27]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.elu = nn.ELU()
        
    def forward(self, x, batched= True):
        if batched :
            x_padded = x[0]
            x_padded = x_padded.cuda()
            tags =x[1]
            tags = tags.cuda()
            x_lens = x[2]

            
            y_lens = x[3]
    
            #pass text through embedding layer
            x_embed = self.dropout(self.embedding(x_padded))

            x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=True, enforce_sorted=False)
            #pass embeddings into LSTM
            output_packed, (hidden, cell) = self.lstm(x_packed)
            output_padded, output_lengths = pad_packed_sequence(output_packed, batch_first=True)
            #we use our outputs to make a prediction of what the tag should be
            outputs = self.elu(output_padded)
            outputs = self.fc(self.dropout(outputs))
        else:
             #pass text through embedding layer
            embedded = self.dropout(self.embedding(x))
            #pass embeddings into LSTM
            outputs, (hidden, cell) = self.lstm(embedded) 
            #we use our outputs to make a prediction of what the tag should be
            outputs = self.elu(outputs)
            outputs = self.fc(self.dropout(outputs))
    
        
        return outputs

In [28]:
INPUT_DIM = len(word_to_index)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 128
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.33
PAD_IDX = word_pad_index

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [29]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

In [30]:
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(23626, 100, padding_idx=0)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#print(f'The model has {count_parameters(model):,} trainable parameters')

In [32]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [33]:
### define optimizer fuction 
optimizer = SGD(model.parameters(), lr=0.05, momentum=0.9)

In [34]:
### define loss function 
TAG_PAD_IDX = tag_to_index['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [35]:
### train the model 
def train(model, data_loader, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for x in data_loader:
       # x = x.cuda()
        tags = x[1]
        tags = tags.cuda()
        optimizer.zero_grad()
        predictions = model(x)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        tags = tags.type(torch.LongTensor) 
        tags = tags.cuda()
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(data_loader),epoch_acc / len(data_loader)

In [36]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.word
            tags = batch.tag
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [38]:
#device = torch.device('cuda:0')
#model.to(device)

In [39]:
# N_EPOCHS = 50

# best_train_loss = float('inf')

# for epoch in range(N_EPOCHS):

#     start_time = time.time()
    
#     train_loss, train_acc = train(model, data_loader, optimizer, criterion, TAG_PAD_IDX)
#     #valid_loss, valid_acc = evaluate(model, corpus.val_iter, criterion, TAG_PAD_IDX)
    
#     end_time = time.time()

#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     if train_loss < best_train_loss:
#         best_train_loss = train_loss
#         torch.save(model.state_dict(), 'tut1-model.pt')
    
#     print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#     #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [40]:
## test on CPU 
device = torch.device("cpu")
model.to(device)

BiLSTMPOSTagger(
  (embedding): Embedding(23626, 100, padding_idx=0)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [41]:
def finder(x):
    token = []
    for w in x:
        if w in word_to_index:
            vec =word_to_index[w]
        else:
            vec = word_to_index["<unk>"]
            
        token.append(vec)
            
    return token 

model.load_state_dict(torch.load('blstm1.pt', map_location=torch.device('cpu')))
def tag_sentence(model, sentence, w_to_ix, t_to_ix):
    
    model.eval()
  
    numericalized_tokens = finder(sentence)
    ## convert it into tensor 
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1)
     # use the model to predict     
    predictions = model(token_tensor, batched=False)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [ix_to_t[t.item()] for t in top_predictions]
    
    return predicted_tags

FileNotFoundError: [Errno 2] No such file or directory: 'blstm1.pt'

In [ ]:
def converter(data):
    train_word = []
    train_tag = []
    i = 0
    for index, word,tag in data:
        if index == 1:
            temp1 = []
            temp2 = []
            temp1.append(word)
            temp2.append(tag)
            
        else:
            temp1.append(word)
            temp2.append(tag)
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            train_word.append(temp1)
            train_tag.append(temp2)
        i += 1
    return train_word, train_tag

In [ ]:
dev_file = pd.read_csv('dev', quoting=csv.QUOTE_NONE, sep=" ", names= ['index', 'word', 'tag'])

dev_array = dev_file.to_numpy()

dev_word, dev_tag = converter(dev_array)

    


In [ ]:
dev_array

In [ ]:
#sentence_tag
#dev_sentences

In [ ]:
def tag_sentence(model, sentence, word_to_index, tag_to_index):
    
    model.eval()
  
    numericalized_tokens = finder(sentence)
    ## convert it into tensor 
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1)
     # use the model to predict     
    predictions = model(token_tensor, batched=False)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [index_to_tag[t.item()] for t in top_predictions]
    
    return predicted_tags

In [ ]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
#from seqeval.metrics import classification_report, f1_score

In [ ]:
#dev_sentences[1]

In [ ]:
whole_pred_tags = []

for s in dev_word:
    
    pred_tags= tag_sentence(model, s,word_to_index, tag_to_index)
    
    whole_pred_tags.append(pred_tags)
    
    

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

# flattenn the nested list 
whole_pred_tags_f = flatten(whole_pred_tags)

dev_actual_tags_f = flatten(dev_tag)

In [ ]:
len(whole_pred_tags_f) ==len(dev_actual_tags_f)

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
from sklearn.metrics import classification_report
labels =  ['B-LOC', 'B-MISC', 'B-ORG',"B-PER","I-LOC","I-MISC","I-ORG","I-PER"]
target = ['B-LOC', 'B-MISC', 'B-ORG',"B-PER","I-LOC","I-MISC","I-ORG","I-PER"]
print(classification_report(dev_actual_tags_f, whole_pred_tags_f, labels = labels, target_names =target ))

In [ ]:

import copy

dev_data_list = copy.deepcopy(dev_stream)

for i in range(len(whole_pred_tags)):

    for i1 in range(len(whole_pred_tags[i])):

        #dev_data_list[i][i1].pop() # pop the actual tag 

        dev_data_list[i][i1].append(whole_pred_tags[i][i1])



def write_out_eva(file_name, data):
    
    with open(file_name,'w') as f:
        for i in range(len(data)):
            if i != 0:
                f.write('\n')
            for (index, w, t,p) in data[i]:
                f.write(str(index))
                f.write(' ')
                f.write(str(w))
                f.write(' ')
                f.write(str(t))
                f.write(' ')
                f.write(str(p))
                f.write('\n')

In [ ]:
write_out_eva('predicted_file_task1',dev_data_list)